# Exercise Sheet 9

## Exercise 1: SHAP

In [1]:
import math
import numpy as np
%matplotlib inline
import pandas as pd

In [2]:
np.random.seed(123456)


### a)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv('fifa.csv')
X, y = df.loc[:, df.columns != 'Man of the Match'], df['Man of the Match']
y = (y == 'Yes')
X = X.select_dtypes([np.number]).copy()
X = X.dropna(axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

classifier_RF = RandomForestClassifier()
classifier_RF.fit(X_train, y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [4]:
# Testing
print(X_test.shape)
print(classifier_RF.predict(X_test.iloc[1:2]))
print(classifier_RF.predict(X_test.iloc[1:10]))
print(classifier_RF.predict_proba(X_test.iloc[1:10]))
print(classifier_RF.predict_proba(X_test.iloc[1:10])[:, 1])

(39, 18)
[False]
[False False  True  True  True False False False False]
[[0.89 0.11]
 [0.7  0.3 ]
 [0.45 0.55]
 [0.23 0.77]
 [0.26 0.74]
 [0.56 0.44]
 [0.81 0.19]
 [0.63 0.37]
 [0.67 0.33]]
[0.11 0.3  0.55 0.77 0.74 0.44 0.19 0.37 0.33]


As a classifier, this model of course produces probabilities as outputs, which are then transformed into a binary prediction. This is not very well suited for Shapley values, since we basically require a numerical or continuous target, in particular for the efficiency axiom to hold up. We will therefore always consider the predicted probabilities in this exercise, although one could also use the predict() function everywhere in the following, which would then be averaged when calculating PDPs or Shapley values.

In [5]:
# Note: Here we predict probabilities, and a standard linear regression is used, not logistic regression.

predict_RF = lambda x : classifier_RF.predict_proba(x)[:, 1]

### b) marginal sampling based SHAP value function

In [6]:
def marginal_vfunc(S, observation, X, predict, nr_samples = 100):
    remainder = set(X.columns) - set(S)
    X_tmp = X.sample(nr_samples, replace=True).copy()
    # Next step is just reshaping and copying the observation nr_samples many times
    observation_df = pd.DataFrame(observation).T.sample(nr_samples, replace=True)
    observation_df = observation_df.reset_index(drop=True)
    X_tmp.loc[:, S] = np.array(observation_df.loc[:, S].copy())
    return np.mean(predict(X_tmp))

In [7]:
# Testing
print(marginal_vfunc(X_test.columns[0:3], X_test.iloc[1,], X_train, predict_RF, nr_samples = 1000))
print(marginal_vfunc(X_test.columns[0:3], X_test.iloc[2,], X_train, predict_RF, nr_samples = 1000))

0.23195999999999997
0.59027


### c) Using this value function in the approximation algo for Shapley values

Necessary functions from the last exercise (copied):

In [8]:
def payoff(coalition):
    # Not necessary in this exercise
    return None

def payoff_diff(member, coalition, v_function=payoff, *args):
    return v_function(coalition + [member], *args) - v_function(coalition, *args)

def shapley_perm_approx(member, population, v_function=payoff, *args, its=100):
    vals = []
    rng = np.random.default_rng()
    for i in range(its):
        permutation = list(rng.permutation(population))
        coalition = permutation[:permutation.index(member)]
        vals.append(payoff_diff(member, coalition, v_function, *args))
    return np.mean(vals)

# Testing and Sanity Check
def all_shapley_values(population, shapley_fct, v_function=payoff, *args):
    return {member:shapley_fct(member, population, v_function, *args) for member in population}

def shapley_test(population, shapley_fct, v_function=payoff, *args):
    shapley_values=all_shapley_values(population, shapley_fct, v_function, *args)
    print("\n".join(f"Player {player}: {shapley_values[player]}" for player in shapley_values))
    sum_ = sum(shapley_values.values())
    print(sum_)
    print(v_function(population, *args))
    print(math.isclose(sum_, v_function(population, *args))) # efficiency axiom

In [9]:
# Testing the approximation algorithm NOTE: This takes a while
print(shapley_perm_approx(X_test.columns[2], X_test.columns, marginal_vfunc, X_test.iloc[2,], X_train, predict_RF))
shapley_test(X_test.columns, shapley_perm_approx, marginal_vfunc, X_test.iloc[2,], X_train, predict_RF)

-0.0076410000000000115
Player Goal Scored: 0.08202400000000001
Player Ball Possession %: -0.070739
Player Attempts: -0.008805000000000004
Player On-Target: 0.012278000000000004
Player Off-Target: -0.015633000000000005
Player Blocked: -0.01725899999999999
Player Corners: -0.013196999999999997
Player Offsides: 0.008061000000000002
Player Free Kicks: -0.017062000000000015
Player Saves: -0.04801000000000001
Player Pass Accuracy %: 0.016870000000000007
Player Passes: -0.07273099999999999
Player Distance Covered (Kms): -0.011760999999999982
Player Fouls Committed: -0.01807
Player Yellow Card: -0.03212000000000001
Player Yellow & Red: 0.00018200000000001104
Player Red: 0.0011640000000000003
Player Goals in PSO: 0.00043599999999998225
-0.20437199999999997
0.3
False


The approximation seems not to be very good, but maybe this is to be expected, since we use a small number of iterations for both the value function (the PD-functions) and the Shapley value approximation.

### d)

In [10]:
from scipy.special import comb
from sklearn.linear_model import LinearRegression

def shap_weights(mask):
    p = mask.shape[1]
    zs = np.sum(mask, axis=1)
    nominator = (p - 1)
    denominator = comb(p, zs) * zs * (p - zs)
    return nominator / denominator
    
def replace_dataset(obs, X, nr_samples):
    X_new = np.array(X.sample(nr_samples, replace=True))
    obs_rep = np.repeat(np.array(obs).reshape(-1, 1), X_new.shape[0], axis=1).T
    mask = np.random.binomial(1, 0.5, np.prod(X_new.shape)).reshape(X_new.shape)
    row_ixs, col_ixs = np.where(mask)
    X_new[row_ixs, col_ixs] = obs_rep[row_ixs, col_ixs]
    X_new = pd.DataFrame(X_new, columns=X.columns)
    return X_new, mask

def shap_data(obs, X, nr_samples, predict):
    X_new, mask = replace_dataset(obs, X, nr_samples)
    weight = shap_weights(mask)
    pred = predict(X_new)
    return mask, pred, weight

def kernel_shap(obs, X, nr_samples, predict):
    mask, pred, weight = shap_data(obs, X, nr_samples, predict)
    lm = LinearRegression()
    lm.fit(mask, pred, weight)
    coef = lm.coef_
    coef = pd.Series(coef, index=X.columns)
    coef['intercept'] = lm.intercept_
    return coef

### e)

In [11]:
# Note: Here we predict probabilities, and a standard linear regression is used, not logistic regression.

shap_vals = kernel_shap(X_test.iloc[0, :], X, 1000, predict_RF)
print(shap_vals)

Goal Scored               0.022826
Ball Possession %        -0.085556
Attempts                 -0.051518
On-Target                 0.018152
Off-Target               -0.018847
Blocked                  -0.005959
Corners                  -0.075750
Offsides                  0.051124
Free Kicks               -0.017505
Saves                    -0.040004
Pass Accuracy %           0.036518
Passes                   -0.012152
Distance Covered (Kms)    0.047336
Fouls Committed           0.009697
Yellow Card               0.047845
Yellow & Red              0.005070
Red                      -0.014575
Goals in PSO             -0.002521
intercept                 0.494431
dtype: float64


The results depend on the metric used to define the neighborhood.